In [ ]:
!pip install chromadb
!pip install langchain-groq


In [ ]:
!pip -q install langchain tiktoken duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 860.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [4]:
import chromadb
import langchain
import langchain_core
import langchain_text_splitters
from langchain_groq import ChatGroq

In [45]:
import wikipedia

# Function to perform search using Wikipedia
def wikipedia_search(query):
    try:
        # Search Wikipedia for the given query
        results = wikipedia.search(query)
        if results:
            # Get the summary of the first result
            summary = wikipedia.summary(results[0], sentences=2)
            return summary
        else:
            return "No results found."
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Test the Wikipedia search function
query = "How can I treat a sprained ankle?"
result = wikipedia_search(query)
print(result)


A sprained ankle (twisted ankle, rolled ankle, turned ankle, etc.) is an injury where sprain occurs on one or more ligaments of the ankle. It is the most commonly occurring injury in sports, mainly in ball sports such as basketball, volleyball, football, pickleball, and tennis.


## Custom Agents with Custom Search




In [7]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain

## Setup Tools

In [46]:
from langchain.tools import Tool

# Define the Wikipedia search tool
tools = [
    Tool(
        name="WikipediaSearch",
        func=wikipedia_search,
        description="Use Wikipedia to answer questions"
    )
]

# Example usage with your agent
# result = agent_executor.invoke("How can I treat a sprained ankle?")
# print(result)


In [47]:
import wikipedia

# Define the Wikipedia wrapper function
def wiki_wrapper(input_text):
    # Search for the input text on Wikipedia
    try:
        results = wikipedia.search(input_text)
        if results:
            # Get the summary of the first result
            summary = wikipedia.summary(results[0], sentences=2)
            return summary
        else:
            return "No results found."
    except Exception as e:
        return f"An error occurred: {str(e)}"


## Prompt Template
This instructs the agent on what to do. Generally, the template should incorporate:

**`tools:`** which tools the agent has access and how and when to call them.

**`intermediate_steps:`** These are tuples of previous (**`AgentAction`**, **`Observation`**) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.

**`input:`** generic user input

In [48]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""

In [49]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [50]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## Custom Output Parser
The output parser is responsible for parsing the LLM output into AgentAction and AgentFinish. This usually depends heavily on the prompt used.

This is where you can change the parsing to do retries, handle whitespace, etc

In [51]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [52]:
output_parser = CustomOutputParser()

## Set up LLM
Choose the LLM you want to use!

In [53]:
llm = ChatGroq(
    temperature=0,
    groq_api_key = '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',
    model="llama-3.1-70b-versatile"

)

response= llm.invoke("who first landed on the Moon, Give exact answer")

In [25]:
response

AIMessage(content="Neil Armstrong was the first person to land on the Moon. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 45, 'total_tokens': 88, 'completion_time': 0.173062131, 'prompt_time': 0.012057512, 'queue_time': 0.005137867000000001, 'total_time': 0.185119643}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-37ac55f4-bc50-4d2a-906f-a59cd097ae74-0', usage_metadata={'input_tokens': 45, 'output_tokens': 43, 'total_tokens': 88})

## Define the stop sequence
This is important because it tells the LLM when to stop generation.

This depends heavily on the prompt and model you are using. Generally, you want this to be whatever token you use in the prompt to denote the start of an Observation (otherwise, the LLM may hallucinate an observation for you).

## Set up the Agent
We can now combine everything to set up our agent

In [54]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [55]:
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)


### What is an Agent Executor?

Agent Executors take an agent and tools and use the agent to decide which tools to call and in what order.



In [56]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,
                                                    tools=tools,
                                                    verbose=True)

### in Debug mode

In [57]:
langchain.debug = True

In [58]:
agent_executor.run("How can I treat a spained ankle?")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How can I treat a spained ankle?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "How can I treat a spained ankle?"
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatGroq] Entering LLM run with input:
{
  "prompts": [
    "Human: Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:\n\nWikipediaSearch: Use Wikipedia to answer questions\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [WikipediaSearch]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the 

"I'm so sorry that I couldn't find the specific treatment options for a sprained ankle. However, I can provide you with some general advice. If you've sprained your ankle, it's essential to rest, ice, compress, and elevate (RICE) the affected area to reduce pain and swelling. You can also use over-the-counter pain relievers, such as acetaminophen or ibuprofen, to manage pain and inflammation. It's also crucial to seek medical attention if you experience severe pain, swelling, or difficulty walking. A healthcare professional can assess the severity of the sprain and provide personalized treatment options. Remember to be patient and gentle with your ankle as it heals, and don't hesitate to reach out if you have any further questions or concerns."

In [59]:
langchain.debug = False

## Adding a Conversation Memory
If you want to add memory to the agent, you’ll need to:

1. Add  **`chat_history`** into the custom prompt

2. Add the memory object to the agent executor when defining it.

In [60]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a compasionate medical professional when giving your final answer. If the condition is serious advise they speak to a doctor.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [61]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [62]:
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [63]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [64]:
from langchain.memory import ConversationBufferWindowMemory

memory=ConversationBufferWindowMemory(k=2)

<ipython-input-64-f238f7e37b46>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferWindowMemory(k=2)


In [65]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory
    )

In [69]:
agent_executor.run("How can I treat a sprained ankle suggest medicine for sprainkled pain?")



> Entering new AgentExecutor chain...
Question: How can I treat a sprained ankle suggest medicine for sprained pain?

Thought: I need to provide information on treating a sprained ankle and suggest some over-the-counter pain relievers that can help manage pain and inflammation.

Action: WikipediaSearch
Action Input: Over-the-counter pain relievers for sprained ankle

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')




Observation:An error occurred: "strain" may refer to: 
Strain (biology)
Strain (chemistry)
Strain (injury)
Strain (mechanics)
Filtration
Percolation
Psychological stress
sieve
Strain
Strain (surname)
Strain (manga)
Strain (music)
Strain (album)
Strain: Strategic Armored Infantry
Strain (film)
Strain, Arkansas
Strain, Missouri
Overwork
S Train (disambiguation)
Strain theory (disambiguation)
Strainer
The Strain (disambiguation)
Stressor
Thought: It seems like the search results are not relevant to the question. I need to try a different approach.

Action: WikipediaSearch
Action Input: Sprained ankle treatment

Observation:A sprained ankle (twisted ankle, rolled ankle, turned ankle, etc.) is an injury where sprain occurs on one or more ligaments of the ankle. It is the most commonly occurring injury in sports, mainly in ball sports such as basketball, volleyball, football, pickleball, and tennis.
Thought: I now have some general information on sprained ankles, but I still need to find s

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')




Observation:An error occurred: "strain" may refer to: 
Strain (biology)
Strain (chemistry)
Strain (injury)
Strain (mechanics)
Filtration
Percolation
Psychological stress
sieve
Strain
Strain (surname)
Strain (manga)
Strain (music)
Strain (album)
Strain: Strategic Armored Infantry
Strain (film)
Strain, Arkansas
Strain, Missouri
Overwork
S Train (disambiguation)
Strain theory (disambiguation)
Strainer
The Strain (disambiguation)
Stressor
Thought: I've tried various search approaches, but I'm still not getting the specific information I need on over-the-counter pain relievers for a sprained ankle. I'll try to provide a general answer based on my previous knowledge.

Thought: I remember from my previous conversation that I suggested the RICE method for treating a sprained ankle, which includes Rest, Ice, Compression, and Elevation. I also mentioned that over-the-counter pain relievers such as acetaminophen or ibuprofen can help manage pain and inflammation.

Thought: I'll provide a genera

"I'm so sorry to hear that you're dealing with a sprained ankle. Fortunately, most sprained ankles can be treated with the RICE method, which stands for Rest, Ice, Compression, and Elevation. \n\nRest: Give your ankle a break and avoid putting weight on it. This will help reduce pain and prevent further injury.\n\nIce: Apply ice to the affected area for 15-20 minutes, 3-4 times a day. This will help reduce pain and inflammation.\n\nCompression: Use an elastic bandage or compression wrap to help reduce swelling.\n\nElevation: Elevate your ankle above the level of your heart to help reduce swelling.\n\nFor pain management, you can try over-the-counter pain relievers such as acetaminophen or ibuprofen. However, please consult with a doctor or pharmacist before taking any medication, especially if you have any underlying medical conditions or allergies.\n\nRemember to be patient and take care of yourself during the recovery process. It may take some time for your ankle to heal, but with pr

In [68]:
agent_executor.run("How long will it take to heal?")



> Entering new AgentExecutor chain...
Question: How long will it take to heal a sprained ankle?

Thought: To answer this question, I need to know the general recovery time for a sprained ankle.

Action: WikipediaSearch
Action Input: Sprained ankle recovery time

Observation:A sprained ankle (twisted ankle, rolled ankle, turned ankle, etc.) is an injury where sprain occurs on one or more ligaments of the ankle. It is the most commonly occurring injury in sports, mainly in ball sports such as basketball, volleyball, football, pickleball, and tennis.
Thought: The observation didn't provide the specific information I was looking for. I need to try a different approach.

Action: WikipediaSearch
Action Input: Sprained ankle recovery time stages

Observation:The Broström operation (or Broström-Gould technique) is a repair of ligaments on lateral ankle. It is designed to address ankle instability.
Thought: It seems like the previous search results didn't provide the information I was looking

'The recovery time for a sprained ankle can vary depending on the severity of the injury, but generally, it can take anywhere from 2-4 weeks for a mild sprain, 6-8 weeks for a moderate sprain, and 3-6 months for a severe sprain.'